Самый первый классификатор, построенный на сайтах из категорий майла и рамблера<br>
На этапе предобработки текста загружаемого сюда файла (html_new_1000words.csv) для текста каждого сайта оставлял только те слова, которые характерны для его категории, тем самым, по ошибке, занес информацию о категориях внутрь датасета <br> По этой причине классификатор выдает сверх высокую точность (около 90%)

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.svm import SVC

In [0]:
df_FullHtml = pd.read_csv('/content/drive/My Drive/html_new_1000words.csv', encoding = 'utf-8-sig')
df_FullHtml = df_FullHtml[df_FullHtml['title']!='none']
df_FullHtml = df_FullHtml[df_FullHtml['title']!='']
df_FullHtml = df_FullHtml[['category','title']].reset_index()
df_FullHtml = df_FullHtml[['category','title']]
df_FullHtml['title'] = df_FullHtml['title'].astype('str')

In [5]:
df2 = df_FullHtml
df4 = df2
df2 = df4[df4['title'].str.split().str.len() > 50]
len(set(df2['category']))

29

In [0]:
df2 = shuffle(df2)
df2 = df2.reset_index()
df2 = df2[['category','title']]

In [0]:
y = df2['category'].to_list()
df2_train, df2_test, y_train, y_test = train_test_split(df2,y, test_size=0.33, random_state=2)

In [0]:
df3 = df2_train.groupby('category')['title'].apply(lambda row: ' '.join(row)).reset_index()

In [0]:
idf_vectorizer = TfidfVectorizer()
tf_idf_matrix_train = idf_vectorizer.fit_transform(df3['title'])
vectorizer = CountVectorizer()
onehot_matrix_train = vectorizer.fit_transform(df2_train['title'])
onehot_matrix_test = vectorizer.transform(df2_test['title'])
result_matrix_train = tf_idf_matrix_train * onehot_matrix_train.T
result_matrix_test = tf_idf_matrix_train * onehot_matrix_test.T

In [0]:
X_train = result_matrix_train.T.toarray()
X_test = result_matrix_test.T.toarray()

In [14]:
model = SVC(C = 1000,gamma = 0.01,kernel = 'rbf')
model.fit(X_train, y_train)

print('X test:')
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

print('X train:')
y_pred = model.predict(X_train)
print(f1_score(y_train, y_pred, average = 'macro'))

X test:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1270
           1       0.85      0.77      0.81       638
           2       0.88      0.89      0.88      1009
           3       0.87      0.93      0.90      3699
           4       0.89      0.92      0.91      2692
           5       0.90      0.90      0.90       153
           6       0.86      0.85      0.86       385
           7       0.89      0.92      0.90      1588
           8       0.94      0.90      0.92       520
           9       0.98      0.93      0.96       229
          10       0.89      0.90      0.90      1994
          11       0.90      0.87      0.89       301
          12       0.90      0.93      0.91       611
          13       0.89      0.88      0.89      1027
          14       0.79      0.69      0.73       353
          15       0.82      0.81      0.82      1833
          16       0.89      0.90      0.89       854
          17       